# Projeto ATD
## 2ºsemestre 2020/2021
### FCTUC - DEI
### Membros:
* Joel Oliveira
* Tomás Mendes


In [ ]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import signal
import scipy as sy
from numpy.fft import fft, fftshift

### Ler dados do acelerómetro para variável
#### (1 variável / experiência )
##### userA_expB (Experiência B do Sujeito A)

In [ ]:
user1_1 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp01_user01.txt"), columns = ['X','Y','Z'])
user1_2 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp02_user01.txt"), columns = ['X','Y','Z'])
user2_1 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp03_user02.txt"), columns = ['X','Y','Z'])
user2_2 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp04_user02.txt"), columns = ['X','Y','Z'])
user3_1 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp05_user03.txt"), columns = ['X','Y','Z'])
user3_2 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp06_user03.txt"), columns = ['X','Y','Z'])
user4_1 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp07_user04.txt"), columns = ['X','Y','Z'])
user4_2 = pd.DataFrame(data = np.loadtxt("Datasets/acc_exp08_user04.txt"), columns = ['X','Y','Z'])

##### Juntar à estrutura com os dados a informação do tempo em minutos de cada experiência

In [ ]:
freq = 50 # Hz
T = 1/freq # sec

user1_1['Time (min)'] = np.arange(0, len(user1_1['X']) * T, T)/60
user1_2['Time (min)'] = np.arange(0, len(user1_2['X']) * T, T)/60 
user2_1['Time (min)'] = np.arange(0, len(user2_1['X']) * T, T)/60 
user2_2['Time (min)'] = np.arange(0, len(user2_2['X']) * T, T)/60 
user3_1['Time (min)'] = np.arange(0, len(user3_1['X']) * T, T)/60 
user3_2['Time (min)'] = np.arange(0, len(user3_2['X']) * T, T)/60 
user4_1['Time (min)'] = np.arange(0, len(user4_1['X']) * T, T)/60 
user4_2['Time (min)'] = np.arange(0, len(user4_2['X']) * T, T)/60

#### Guardar os dados das experiências num ficheiro .dat, para ler mais tarde

In [ ]:
users_exp = [user1_1, user1_2, user2_1, user2_2, user3_1, user3_2, user4_1, user4_2]
pd.to_pickle(users_exp, "users.dat")

### Verificação dos dados presentes no DataSet, através da função 'info()' das DataFrame da livraria Pandas.
#### Esta função devolve informações sobre a informação presente na DataFrame. É devolvida uma string com a quantidade de valores presente não nula, a quantidade de valores presente com valor nulo e ainda o tipoda informação presente em cada coluna. 
#### Assim, é possível verificar que o DataSet está bastante completo, sem falhas de informação, e o tipo de dados existente nele. No caso dados do tipo float.

In [ ]:
for i in range(len(users_exp)):
    print("//" + "_"*9 + f"Sujeito {i//2+1} Experiencia {i%2+1}" + "_"*9 + "\\\\", end = '\n\n')
    users_exp[i].info()
    print("\n")
    

In [ ]:
for i in range(len(users_exp)):
    print("//" + "_"*15 + f"Sujeito {i//2+1} Experiencia {i%2+1}" + "_"*15 + "\\\\", end='\n\n')
    print(users_exp[i].describe())
    print("\n")


#### Leitura dos dados inseridos no ficheiro 'users.dat', anteriormente.
##### Este ficheiro contém um array com as informações de cada experiência inseridas numa DataFrame

In [ ]:
users_exp = pd.read_pickle("users.dat")
user1_1, user1_2, user2_1, user2_2, user3_1, user3_2, user4_1, user4_2 = users_exp

### Atividades Dinâmicas:
#### 1. WALKING
#### 2. WALKING_UPSTAIRS
#### 3. WALKING_DOWNSTAIRS

### Atividades Estáticas:
#### 1. SITTING
#### 2. STANDING
#### 3. LAYING

### Atividades de Transição:
#### 1. STAND_TO_SIT
#### 2. SIT_TO_STAND
#### 3. SIT_TO_LIE
#### 4. LIE_TO_SIT
#### 5. STAND_TO_LIE
#### 6. LIE_TO_STAND

#### Função para escrever manualmente as atividades nos plots

In [ ]:
def plot_with_labels(upxvalues, downxvalues, up, down, user, index):
    '''
    Plots the X, Y, Z axis from the accelerometer
            Parameters:
                    upxvalues (list): ordenadas para impressão de up (labels a colocar em cima)
                    downxvalues (list): ordenadas para impressão de down (labels a colocar em baixo)
                    up (list): lista com as labels das atividades
                    down (list): list com as labels das atividades
                    user (pandas.dataFrame): dataFrame com os dados do user a imprimir.
                    index (int): numero da experiência, de modo a imprimir o Numero do Sujeito, bem como da sua experiência
    '''
    figure, subplots = plt.subplots(nrows=3, ncols=1, figsize=(16,8))
    figure.suptitle(f"Valores Obtidos Pelo Acelerómetro na Experiência {index%2+1} do Sujeito {index//2+1}", fontsize = 'xx-large')
    
    #X
    subplots[0].plot( user['Time (min)'], user['X'], lw = 0.3)
    subplots[0].set_xlabel("Time (min)")
    subplots[0].set_ylabel("ACC_X")
    subplots[0].set_xlim(0, max(user['Time (min)']))
    
    #Y
    subplots[1].plot( user['Time (min)'], user['Y'], lw = 0.3)
    subplots[1].set_xlabel("Time (min)")
    subplots[1].set_ylabel("ACC_Y")
    subplots[1].set_xlim(0, max(user['Time (min)']))
    
    #Z
    subplots[2].plot( user['Time (min)'], user['Z'], lw = 0.3)
    subplots[2].set_xlabel("Time (min)")
    subplots[2].set_ylabel("ACC_Z")
    subplots[2].set_xlim(0, max(user['Time (min)']))
    
    xmin,xmax = subplots[0].get_xlim()
    
    for i in range(len(upxvalues)):
        yv = 0.95 if i % 2 == 0 else 0.90
        frase = up[i]
        xv = upxvalues[i]/xmax
        
        subplots[0].annotate(xycoords = 'axes fraction', text = frase, xy = (xv,yv), textcoords = subplots[0].transAxes, xytext = (xv,yv), fontweight='bold')
        subplots[1].annotate(xycoords = 'axes fraction', text = frase, xy = (xv,yv), textcoords = subplots[1].transAxes, xytext = (xv,yv), fontweight='bold')
        subplots[2].annotate(xycoords = 'axes fraction', text = frase, xy = (xv,yv), textcoords = subplots[2].transAxes, xytext = (xv,yv), fontweight='bold')
        
    yv = 0.01 
    for i in range(len(downxvalues)):
        frase = down[i]
        xv = downxvalues[i]/xmax
        
        subplots[0].annotate(xycoords = 'axes fraction', text = frase, xy = (xv,yv), textcoords = subplots[0].transAxes, xytext = (xv,yv), fontweight='bold')
        subplots[1].annotate(xycoords = 'axes fraction', text = frase, xy = (xv,yv), textcoords = subplots[1].transAxes, xytext = (xv,yv), fontweight='bold')
        subplots[2].annotate(xycoords = 'axes fraction', text = frase, xy = (xv,yv), textcoords = subplots[2].transAxes, xytext =(xv,yv), fontweight='bold')
    figure.tight_layout()

## Visualização das atividades para todas as experiências do nosso dataset

In [ ]:
downintervals = [0.08, 0.44, 0.78,  1.2, 1.6, 1.9, 2.5,  3.2, 4.4, 5, 5.6]
upintervals = [0.39, 0.7, 1.1, 1.5, 1.9, 2.1, 2.8, 3.55, 4.7, 5.25, 5.8  ]

downlabels = ['STAND', 'SIT', 'STAND','LAY', 'SIT', 'LAY', 'W', 'W','W_D', 'W_D', 'W_D']
uplabels = ['STAND_SIT', 'SIT_STAND', 'STAND_LIE', 'LIE_SIT', 'SIT_LIE', 'LIE_STAND', 'W', 'W', 'W_U', 'W_U', 'W_U' ]

plot_with_labels(upintervals, downintervals, uplabels, downlabels, user1_1, 0)

In [ ]:
downintervals = [ 0.08, 0.45, 0.75, 1.3, 1.55, 1.95, 2.5, 3.4, 4.4, 5, 5.6]
upintervals = [ 0.40, 0.7, 1.15, 1.5, 1.8, 2.1, 2.9, 3.8, 4.75, 5.3, 5.9 ]

downlabels = [ 'STAND', "SIT", "STAND", "LAY", "SIT", "LAY", "W", "W", "W_D", "W_D", "W_D" ]
uplabels = [ "STAND_SIT", "SIT_STAND", "STAND_LIE", "LIE_SIT", "SIT_LIE", "LIE_STAND", "W", "W", "W_U", "W_U", "W_U" ]

plot_with_labels(upintervals, downintervals, uplabels, downlabels,user1_2, 1)

In [ ]:
downintervals = [ 0.12, 0.6, 1, 1.5, 1.85, 2.2, 2.5, 3.4, 4.25, 4.9, 5.4 ]
upintervals = [ 0.45, 0.87, 1.3, 1.6, 2.05, 2.5, 2.9, 3.95, 4.6, 5.15 ]

downlabels = [ "STAND", "SIT", "STAND", "LAY", "SIT", "LIE", "W", "W", "W_U", "W_U", "W_U"]
uplabels = [ "STAND_SIT", "SIT_STAND", "STAND_LIE", "LIE_SIT", "SIT_LIE", "LIE_STAND", "W", "W_D", "W_D", "W_D"]

plot_with_labels(upintervals, downintervals, uplabels, downlabels, user2_1, 2)

In [ ]:
downintervals = [ 0.2, 0.6, 0.8, 1.2, 1.5, 1.9, 2.3, 2.95, 3.8, 4.4, 4.9 ]
upintervals = [ 0.4, 0.7, 1.1, 1.4, 1.7, 2.05, 2.6, 3.5, 4.1, 4.65 ]

downlabels = [ "STAND", "SIT", "STAND", "LAY", "SIT", "LIE", "W", "W", "W_U", "W_U", "W_U" ]
uplabels = [ "STAND_SIT",  "SIT_STAND", "STAND_LIE", "LIE_SIT", "SIT_LIE", "LIE_STAND", "W", "W_D", "W_D", "W_D" ]

plot_with_labels(upintervals, downintervals, uplabels, downlabels, user2_2, 3)

In [ ]:
downintervals = [ 0.1, 0.55, 0.9, 1.3, 1.7, 2.2, 2.6, 3.6, 4.7, 5.3, 6, 6.6]
upintervals = [ 0.45, 0.7, 1.1, 1.6, 1.95, 2.3, 3, 4.3, 5.05, 5.7, 6.3 ]

downlabels = [ "STAND", "SIT", "STAND", "LAY", "SIT", "LAY", "W", "W", "W_U", "W_U", "W_U", "W_U"]
uplabels = [ "STAND_SIT", "SIT_STAND", "STAND_LIE", "LIE_SIT", "SIT_LIE", "LIE_STAND", "W", "W_D", "W_D", "W_D", "W_D" ]

plot_with_labels(upintervals, downintervals, uplabels, downlabels, user3_1, 4)

In [ ]:
downintervals = [ 0.2, 0.7, 1, 1.5, 1.9, 2.3, 2.6, 3.5, 4.3, 4.9, 5.45 ]
upintervals = [ 0.6, 0.8, 1.3, 1.7, 2.1, 2.5, 3, 4, 4.6, 5.2 ]

downlabels = [ "STAND", "SIT", "STAND", "LAY", "SIT", "LAY", "W", "W", "W_U", "W_U", "W_U", "W_U"]
uplabels = [ "STAND_SIT", "SIT_STAND", "STAND_LIE", "LIE_SIT", "SIT_LIE", "LIE_STAND", "W", "W_D", "W_D", "W_D", "W_D" ]

plot_with_labels(upintervals, downintervals, uplabels, downlabels, user3_2, 5)

In [ ]:
downintervals = [ 0.15, 0.6, 0.9, 1.3, 1.7, 2.1, 2.45, 3.3, 4.3, 4.9, 5.45 ]
upintervals = [ 0.4, 0.7, 1.1, 1.6, 1.9, 2.3, 2.9, 3.8, 4.6, 5.2 ]

downlabels = [ "STAND", "SIT", "STAND", "LAY", "STAND", "LAY", "W", "W", "W_U", "W_U", "W_U", "W_U"]
uplabels = [ "STAND_SIT", "SIT_STAND", "STAND_LIE", "LIE_STAND", "STAND_LIE", "LIE_STAND", "W", "W_D", "W_D", "W_D", "W_D" ]

plot_with_labels(upintervals, downintervals, uplabels, downlabels, user4_1, 6)

In [ ]:
downintervals = [ 0.15, 0.6, 0.9, 1.3, 1.6, 2, 2.4, 3.1, 3.9, 4.4, 4.9 ]
upintervals = [ 0.4, 0.7, 1.1, 1.6, 1.9, 2.2, 2.7, 3.6, 4.2, 4.6 ]

downlabels = ["STAND", "SIT", "STAND", "LAY", "STAND", "LAY", "W", "W", "W_U", "W_U", "W_U", "W_U"]
uplabels = [  "STAND_SIT", "SIT_STAND", "STAND_LIE", "LIE_STAND", "STAND_LIE", "LIE_STAND", "W", "W_D", "W_D", "W_D", "W_D"  ]

plot_with_labels(upintervals, downintervals, uplabels, downlabels, user4_2, 7)

## cenas do 3

In [ ]:
#uma segmentação das informações do user, num intervalo [*xi*, *xf*[
def ufrag(user, xi, xf):
    return user[ (xi<=user['Time (min)']) & (user['Time (min)'] < xf) ].reset_index().drop("index", axis = 1)

In [ ]:
### Dá plot às informações dos 3 eixos de uma fragmentação do utilizador nos subplots *subplots*, com a cor *c*
## Para ajudar a análise também dá plot a uma linha com a média dos valores, bem como da média +/- 3* desvio padrão
def plot_walk(subplots, user, c):
    # subplots -- array com 3 subplots
    # c -- string com a cor
    # user -- DataFrame com o user
    xmean = user['X'].mean()
    xstd = user['X'].std()
    user.plot( x = 'Time (min)', y = 'X', ax = subplots[0], color = c )
    subplots[0].plot( user['Time (min)'], np.full( (len(user['X']), ), xmean ), 'k:')
    subplots[0].plot( user['Time (min)'], np.full( (len(user['X']), ), xmean + 3*xstd ), 'r--')
    subplots[0].plot( user['Time (min)'], np.full( (len(user['X']), ), xmean - 3*xstd ), 'r--')
    subplots[0].set_xlabel("Time (min)")
    subplots[0].set_ylabel("ACC_X")
    
    
    ymean = user['Y'].mean()
    ystd = user['Y'].std()
    user.plot( x = 'Time (min)', y = 'Y', ax = subplots[1], color = c )
    subplots[1].plot( user['Time (min)'], np.full( (len(user['Y']), ), ymean ), 'k:')
    subplots[1].plot( user['Time (min)'], np.full( (len(user['Y']), ), ymean + 3*ystd ), 'r--')
    subplots[1].plot( user['Time (min)'], np.full( (len(user['Y']), ), ymean - 3*ystd ), 'r--')
    subplots[1].set_xlabel("Time (min)")
    subplots[1].set_ylabel("ACC_Y")
    
    zmean = user['Z'].mean()
    zstd = user['Z'].std()
    user.plot( x = 'Time (min)', y = 'Z', ax = subplots[2], color = c )
    subplots[2].plot( user['Time (min)'], np.full( (len(user['Z']), ), zmean ), 'k:')
    subplots[2].plot( user['Time (min)'], np.full( (len(user['Z']), ), zmean + 3*zstd ), 'r--')
    subplots[2].plot( user['Time (min)'], np.full( (len(user['Z']), ), zmean - 3*zstd ), 'r--')
    subplots[2].set_xlabel("Time (min)")
    subplots[2].set_ylabel("ACC_Z")

In [ ]:
### Dá plot às informações nos eixos das fragmentações do utilizador
def plot_user_walks(user_walks):
    # user_walks -- array com as fragmentações dos dados do utilizador // não deve exceder 4 fragmentações.
    figure, subplots = plt.subplots(nrows = 6, ncols= 2, figsize = (14,10))
    colors = ['royalblue', 'maroon', 'darkorange', 'olive']
    
    for i in range(len(user_walks)):
        if i < 2:
            plot_walk(subplots[:3,i], user_walks[i], colors[i] )
            subplots[0,i].set_title(f"WALK {i+1}")
        else:
            plot_walk(subplots[3:,i-2], user_walks[i], colors[i] )
            subplots[3,i-2].set_title(f"WALK {i+1}")
    figure.tight_layout()

In [ ]:
def set_between_std(user):
    xmean = user['X'].mean()
    xstd = user['X'].std()
    user['X'] = user['X'].map(lambda x: (xmean + 2.5*xstd) if (x > xmean + 3*xstd) 
                              else ( (xmean - 2.5*xstd) if (x < xmean - 3*xstd) 
                                    else x ) )
    
    ymean = user['Y'].mean()
    ystd = user['Y'].std()
    user['Y'] = user['Y'].map(lambda x: (ymean + 2.5*ystd) if (x > ymean + 3*ystd) 
                              else ( (ymean - 2.5*ystd) if (x < ymean - 3*ystd) 
                                    else x ) )
    
    zmean = user['Z'].mean()
    zstd = user['Z'].std()
    user['Z'] = user['Z'].map(lambda x: (zmean + 2.5*zstd) if (x > zmean + 3*zstd) 
                              else ( (zmean - 2.5*zstd) if (x < zmean - 3*zstd) 
                                    else x ) )

In [ ]:
def detrend_user_walk(user):
    new = user.copy()
    new['X'] = signal.detrend(user['X'])
    new['Y'] = signal.detrend(user['Y'])
    new['Z'] = signal.detrend(user['Z'])
    return new

In [ ]:
def plot_period_walk_freqs(walks, axe, freqs, size, N, index, adjust = True):
    user_walks = [ ]
    for j in range(len(walks)):
        user_walks.append( [] )
        for i in range(size//N):
            user_walks[-1].append( np.array(walks[j][axe])[i*step:(i+1)*step] )
    
    dfts = []
    for j in range(len(walks)):
        dfts.append(fftshift(fft( user_walks[j][index] )))
        if adjust:
            dfts[-1][ np.abs(dfts[-1]/N) < 0.01 ] = 0
    
    figure, subplots = plt.subplots(nrows = 2, ncols= 2, figsize = (16,8))
    colors = ['royalblue', 'maroon', 'darkorange', 'olive']
    
    for i in range(len(user_walks)):
        if i < 2:
            subplots[0,i].stem(freqs, np.abs(dfts[i]))
            subplots[0,i].set_title(f"PERIOD {index} of WALK {i+1}, in AXE {axe}")
        else:
            subplots[1,i-2].stem(freqs, np.abs(dfts[i]))
            subplots[1,i-2].set_title(f"PERIOD {index} of WALK {i+1} in AXE {axe}")
    figure.tight_layout()

## Fragmentação da atividade dinâmica WALKING
### Primeiro Seccionar os intervalos da realização da atividade, de modo a poder operar apenas sobre eles

### AVALIAR A PRIMEIRA EXPERIÊNCIA DO PRIMEIRO VOLUNTÁRIO

In [ ]:
unow = user1_1
walks_user1_1 = [ ufrag(unow, 2.56, 2.693), ufrag(unow, 2.955, 3.088), ufrag(user1_1, 3.355, 3.488), ufrag(user1_1, 3.715, 3.848)]
walks_user1_1_detrended = []
for i in walks_user1_1:
    walks_user1_1_detrended.append( detrend_user_walk(i) )
plot_user_walks(walks_user1_1_detrended)

### Pela análise visual dos gráficos acima, a ação aparenta ter uma periodicidade próxima dos 0.019 s

In [ ]:
step = 57
plt.figure(figsize = (16,8))
plt.subplot(221)
for i in range(1,399//step + 1):
    plt.plot(walks_user1_1_detrended[0]['Time (min)'][(i-1)*step: i * step], walks_user1_1_detrended[0]['Z'][(i-1)*step: i * step])
    
plt.subplot(222)
for i in range(1,399//step + 1):
    plt.plot(walks_user1_1_detrended[1]['Time (min)'][(i-1)*step: i * step], walks_user1_1_detrended[1]['Z'][(i-1)*step: i * step])
    
plt.subplot(223)
for i in range(1,399//step + 1):
    plt.plot(walks_user1_1_detrended[2]['Time (min)'][(i-1)*step: i * step], walks_user1_1_detrended[2]['Z'][(i-1)*step: i * step])
    
plt.subplot(224)
for i in range(1,399//step + 1):
    plt.plot(walks_user1_1_detrended[3]['Time (min)'][(i-1)*step: i * step], walks_user1_1_detrended[3]['Z'][(i-1)*step: i * step])

In [ ]:
step = 57
plt.figure(figsize = (16,8))

plt.subplot(221)
for i in range(399//step):
    values = np.array(walks_user1_1_detrended[0]['Z'])[i*step:(i+1)*step]
    plt.plot(range(len(values)), values )

plt.subplot(222)
for i in range(399//step):
    values = np.array(walks_user1_1_detrended[1]['Z'])[i*step:(i+1)*step]
    plt.plot(range(len(values)), values )

plt.subplot(223)    
for i in range(399//step):
    values = np.array(walks_user1_1_detrended[1]['Z'])[i*step:(i+1)*step]
    plt.plot(range(len(values)), values )

plt.subplot(224)
for i in range(399//step):
    values = np.array(walks_user1_1_detrended[1]['Z'])[i*step:(i+1)*step]
    plt.plot(range(len(values)), values )

### Como é visível pelos gráficos acima, parece legítimo considerar 57 como um valor aproximado de N para o intervalo criado, havendo uma sobreposição de todas as divisões e aproximando se bastante de um período no gráfico acima. Visualmente, no gráfico anterior, também se verifica que as diferentes cores se assemelham a um período

In [ ]:
N = 57
fs = 50
if N%2==0:
    f = np.arange( -fs/2, fs/2 - fs/2/N + fs/N, fs/N) 
else:
    f = np.arange( -fs/2 + fs/2/N, fs/2 - fs/2/N + fs/N, fs/N)

In [ ]:
plot_period_walk_freqs(walks_user1_1_detrended, 'Z', f, 399, N, 0, False)

In [ ]:
values = []
for i in range(399//N):
    values.append(np.array(walks_user1_1_detrended[0]['Z'])[i*step:(i+1)*step])

In [ ]:
#plt.stem(f, abs(fftshift(fft(values[0]* hann_window))))

In [ ]:
plt.stem(f, abs(fftshift(fft(values[0]))))